## Deployment Services

### The model will be deployed as a service that consist of:

#### A script to load the model and return predictions for submitted data.
#### An environment in which the script will be run.

## 1. Create the entry script 
### (sometimes referred to as the scoring script) for the service as a Python (.py) file. It must include two functions:

 #### -  init(): Called when the service is initialized.
 #### -  run(raw_data): Called when new data is submitted to the service.

In [1]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

## 2. Creating an Environment


### Conda configuration file

#### Your service requires a Python environment in which to run the entry script, which you can configure using Conda configuration file. An easy way to create this file is to use a CondaDependencies class to create a default environment (which includes the azureml-defaults package and commonly-used packages like numpy and pandas), add any other required packages, and then serialize the environment to a string and save it:

In [2]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

FileNotFoundError: [Errno 2] No such file or directory: 'service_files/env.yml'

### Combining the Script and Environment in an InferenceConfig

#### After creating the entry script and environment configuration file, you can combine them in an InferenceConfig for the service like this:

In [3]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

ERROR - source directory /Users/mohitsewak/temp/Azure/Examples/mnist_pipeline/service_files doesn't exist. 



WebserviceException: WebserviceException:
	Message: source directory /Users/mohitsewak/temp/Azure/Examples/mnist_pipeline/service_files doesn't exist. 
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "source directory /Users/mohitsewak/temp/Azure/Examples/mnist_pipeline/service_files doesn't exist. "
    }
}

## 3. Define a Deployment Configuration

### Now that you have the entry script and environment, you need to configure the compute to which the service will be deployed. 
#### - If you are deploying to an AKS cluster, you must create the cluster and a compute target for it before deploying:

#### - The code to configure an ACI deployment is similar, except that 
 - you do not need to explicitly create an ACI compute target, 
 - and you must use the deploy_configuration class from the azureml.core.webservice.AciWebservice namespace. 

#### - Similarly, you can use the azureml.core.webservice.LocalWebservice namespace to configure a local Docker-based service.

In [4]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

NameError: name 'ws' is not defined

#### With the compute target created, you can now define the deployment configuration, which sets the target-specific compute specification for the containerized deployment:

In [5]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 0.25)

## 4. Deploy the Model
#### After all of the configuration is prepared, you can deploy the model. 
 - The easiest way to do this is to call the deploy method of the Model class, like this:

- For ACI or local services, you can omit the deployment_target parameter (or set it to None).

In [6]:
from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

NameError: name 'ws' is not defined

##### For more info, Read the docs.... 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where